# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(mailto = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 500,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&obj-id.prefix=10.1101&source=twitter&rows=500&from-occurred-date=2021-08-23&until-occurred-date=2021-08-30


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

15635 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(mailto = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-08-23&until-occurred-date=2021-08-30
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=02d82d0a-8a8d-45ff-9c0b-38a52b301f45&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-08-23&until-occurred-date=2021-08-30
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=f4af1ed3-c115-4dbd-91b8-b6066671c27a&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-08-23&until-occurred-date=2021-08-30
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&curso

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-07-29.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load preprint_tweets_2021-08-09.csv
failed to load preprint_tweets_2021-08-23.csv
failed to load preprint_tweets_2021-08-02.csv
failed to load preprint_tweets_2021-08-16.csv
failed to load .ipynb_checkpoints
failed to load preprint_tweets_2021-07-26.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.07.13.452194...
REST API query complete  200
REST API query started for 10.1101/2021.08.15.21262067...
REST API query complete  200
REST API query started for 10.1101/2021.07.31.21261387...
REST API query complete  200
REST API query started for 10.1101/2021.04.19.21255739...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/2021.08.19.456982...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.08.12.21261991...
REST API query complete  200
REST API query started for 10.1101/2021.08.08.21261768...
REST API query complete  200
REST API query started for 10.1101/2021.04.23.21255973...
REST API query complete  200
REST API query started for 10.1101/2021.05.18.21257385...
REST API query complete  200
REST API query started for 10.1101/2021.07.10.2

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.08.20.457153,56,bioRxiv,Genomics,False,"Loops, TADs, Compartments, and Territories are...","[{'name': 'Jacob T. Sanders'}, {'name': 'Rosel...",<p>Layers of genome organization are becoming ...,2021-08-23
1,10.1101/2021.08.25.457692,24,bioRxiv,Immunology,True,The Lambda variant of SARS-CoV-2 has a better ...,"[{'name': 'Haolin Liu'}, {'name': 'Pengcheng W...",<p>The newly emerging variants of SARS-CoV-2 f...,2021-08-26
2,10.1101/2021.08.25.457694,20,bioRxiv,Developmental Biology,False,SOX transcription factors direct TCF-independe...,"[{'name': 'Shreyasi Mukherjee'}, {'name': 'Dav...",<p>WNT/ß-catenin signaling regulates gene expr...,2021-08-26
3,10.1101/2021.08.24.457563,9,bioRxiv,Evolutionary Biology,False,The Evolution of Nest Site Specialization and ...,"[{'name': 'Rosana Zenil-Ferguson'}, {'name': '...",<p>A long-standing hypothesis in evolutionary ...,2021-08-25
4,10.1101/2021.08.20.21261814,9,medRxiv,Genetic and Genomic Medicine,False,Comprehensive genetic analysis of the human li...,"[{'name': 'Gemma Cadby'}, {'name': 'Corey Gile...",<p>We integrated lipidomics and genomics to un...,2021-08-25
5,10.1101/2021.08.23.457408,9,bioRxiv,Immunology,True,Deep immune profiling of the maternal-fetal in...,"[{'name': 'Suhas Sureshchandra'}, {'name': 'Mi...",<p>Pregnant women are an at-risk group for sev...,2021-08-23
6,10.1101/2021.08.22.457251,8,bioRxiv,Neuroscience,False,Modeling movie-evoked human brain activity usi...,[{'name': 'Shinji Nishimoto'}],"<p>In this paper, the process of building a mo...",2021-08-23
7,10.1101/2021.08.24.457556,7,bioRxiv,Plant Biology,False,Interactions between a mechanosensitive channe...,"[{'name': 'Yanbing Wang'}, {'name': 'Joshua Co...",<p>Cells employ multiple systems to maintain c...,2021-08-25
8,10.1101/2021.08.25.457601,7,bioRxiv,Molecular Biology,False,Plant Polymerase IV sensitizes chromatin throu...,"[{'name': 'Vivek Hari Sundar G'}, {'name': 'Ch...",<p>Heterochromatin is the predominant architec...,2021-08-25
9,10.1101/2021.08.22.457032,7,bioRxiv,Systems Biology,False,A multiscale spatiotemporal model explains suc...,"[{'name': 'David M. Versluis'}, {'name': 'Ruud...",<p>The human intestinal microbiome starts to f...,2021-08-23


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

17 preprints (including 2 covering SARS-CoV-2, 16 from bioRxiv and 1 from medRxiv) published in the last 7 days before 2021-08-30 had been tweeted at least 3 times (maximum 56).